
## Modified from CIS 530 Homework Option 2 - Spring 2021




### Installing the Huggingface🤗 transformer package

In [1]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "0"
!pip install transformers
!pip3 install sentencepiece
!pip install madgrad
type_name = "5Cat"

     |████████████████████████████████| 2.1MB 9.6MB/s 
     |████████████████████████████████| 3.3MB 34.8MB/s 
     |████████████████████████████████| 901kB 37.3MB/s 
     |████████████████████████████████| 1.2MB 10.8MB/s 


In [2]:
train_set_name = "Elec"

### Import the important packages that we need

In [3]:
import torch 
import numpy as np
import pandas as pd

### Mount your google drive 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
def get_metrics(true_labels, pred_labels):
    f1 = f1_score(true_labels, pred_labels, average='macro')
    acc = accuracy_score(true_labels, pred_labels)
    rec = recall_score(true_labels, pred_labels, average='macro')
    prec = precision_score(true_labels, pred_labels, average='macro')
    return f1, acc, rec, prec

### Download the Review Datset

Note that with the default code, the files are not saved in your google drive, which means they will get deleted after the session close. You can either re-run this cell for each new colab session, or you can save it to the mounted drive at `/content/drive`

In [6]:
import json
import gzip

def parse(path):
  true = True
  false = False
  g = gzip.open(path, 'r')
  for l in g:
    yield json.dumps(eval(l))

def strict(path):
  max_reviews = 1689188
  iter = 5000
  counter = 0 
  f_path = path + ".json.gz"
  true = True
  false = False
  f = open(path+".json", 'w')
  # f.write("[")
  writable = "["
  # writable.join

  for l in parse(f_path):
    writable = writable + l + ',\n'
    counter = counter + 1
    if (counter % iter) == 0:
      print(f"On line{counter}, {counter/max_reviews} % done")
  writable = writable[:-2]
  writable = writable + "]"
  f.write(writable)

def fast_parse(path):
  # path = path + ".json.gz"
  max_reviews = 1689188
  iter = 5000
  counter = 0 
  all_reviews = []
  g = open(path, 'r')
  for line in g:
    d = json.loads(line)
    all_reviews.append(d)
    if (counter % iter) == 0:
      print(f"On line{counter}, {counter/max_reviews} % done")
    counter = counter + 1
  return all_reviews

In [7]:
#Csvs

train_set = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_train.csv')
dev_set = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test.csv')
experiment_set = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.csv')
train_set = train_set.to_dict(orient='records')
dev_set = dev_set.to_dict(orient='records')
experiment_set = experiment_set.to_dict(orient='records')

# #Jsons

# train_set = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_train_binaryposneg.json')
# dev_set = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test_binaryposneg.json')
# experiment_set = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test_binaryposneg.json')
  


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Load the dataset and see what it looks like

For now let's first load the training dataset and see what it looks like. We will worry about the dev/test sets later...


In [8]:
print(train_set[0]['title_plus_review'])
print("Number of review in training set: {}".format(len(train_set)))
print("Here's how one of the example looks like: {}".format(json.dumps(train_set[100])))

ok the camera is good ,  but the grip/stand could be a little better .   when i try to clip on the monitor ,  it falls off . 
Number of review in training set: 249828
Here's how one of the example looks like: {"Unnamed: 0": 100, "reviewText": "DO NOT BUY THIS!! It completely stopped working after 7 months, and went in and out of connectivity the whole 7 months that it did work", "summary": "DO NOT BUY", "overall": 1.0, "title_plus_review": "do not buy do not buy this !  !  it completely stopped working after 7 months ,  and went in and out of connectivity the whole 7 months that it did work", "pos_neg": 0, "neutrality": 1, "num_words": 29, "nw": 26}


In [9]:
import random

training_reviews = []
id = 0;
for  review in train_set:
  if 'title_plus_review' not in review:
    continue
  training_reviews.append({
            "review_id": id,
            "review_text": str(review['title_plus_review']).encode('utf-8'),
            "label": review['overall']
        })
  
  id = id + 1
print("Number of reviews for training: {}".format(len(training_reviews)))

dev_reviews = []
id = 0;
for review in dev_set:
  #no Review text
  if 'title_plus_review' not in review:
    continue
  dev_reviews.append({
            "review_id": id,
            "review_text": str(review['title_plus_review']).encode('utf-8'),
            "label": review['overall']
        })
  
  id = id + 1
print("Number of reviews for dev: {}".format(len(dev_reviews)))

experiment_reviews = []
id = 0;
for review in experiment_set:
  #no Review text
  if 'title_plus_review' not in review:
    continue
  experiment_reviews.append({
            "review_id": id,
            "review_text": str(review['title_plus_review']).encode('utf-8'),
            "label": review['overall']
        })
  
  id = id + 1
print("Number of reviews for experiment: {}".format(len(experiment_reviews)))

Number of reviews for training: 249828
Number of reviews for dev: 49954
Number of reviews for experiment: 49943


### Load Pretrained BERT Model
 

You can search for the available models [here](https://huggingface.co/models?search=bert).

You can find more examples of different use cases for BERT in the transformer github repo README -- https://github.com/huggingface/transformers


In [10]:
from transformers import InputExample
from transformers import (WEIGHTS_NAME, BertConfig,
                          BertForSequenceClassification, BertTokenizer)
from transformers import glue_convert_examples_to_features as convert_examples_to_features
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
import tqdm

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,  TensorDataset)


bert_model_type = 'bert-base-uncased' 
bert_model = BertForSequenceClassification.from_pretrained(bert_model_type, num_labels = 5)
config = BertConfig.from_pretrained(bert_model_type)
tokenizer = BertTokenizer.from_pretrained(bert_model_type, num_labels = 5)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### Convert examples to BERT input features



In [11]:
relevance_label_mapping = {
    1.0: 1,
    2.0: 2,
    3.0: 3,
    4.0: 4,
    5.0: 5
}

def convert_sentence_pair_to_tensor_input(sentence_reviews, label_mapping):

    # STEP 1: convert each sentence 
    input_examples = []
    for review in sentence_reviews:
        current_label = review["label"] if "label" in review else print("dang")
        input_examples.append(
            InputExample(guid=review["review_id"], 
                         text_a=str(review["review_text"]), 
                         label=label_mapping[current_label])
        )

    label_list = [val for _, val in label_mapping.items()]
    features = convert_examples_to_features(input_examples,
                                                   tokenizer,
                                                   label_list=label_list,
                                                   max_length = 128,  
                                                   output_mode="classification")
    
    input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    labels = torch.tensor([f.label for f in features], dtype=torch.long)
    dataset = TensorDataset(input_ids, attention_mask, token_type_ids, labels)

    return dataset

In [12]:
train_dataset = convert_sentence_pair_to_tensor_input(training_reviews, relevance_label_mapping)

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [13]:
dev_dataset = convert_sentence_pair_to_tensor_input(dev_reviews, relevance_label_mapping)

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [14]:
experiment_dataset = convert_sentence_pair_to_tensor_input(experiment_reviews, relevance_label_mapping)

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


### Choose your hyperparameters + model output directory


In [15]:
HYPER_PARAMS = {
    "num_training_epoch": 3,
    "learning_rate": 5e-5,        
    "training_batch_size": 32,    
    "eval_batch_size": 8,
    "max_grad_norm": 1.0,
    "num_warmup_steps": 0.1,
    "dropout_prob": 0.1
}


In [16]:
torch.cuda.manual_seed(42)
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

bert_model.hidden_dropout_prob = HYPER_PARAMS['dropout_prob']
bert_model.to("cuda")
optimizer = AdamW(bert_model.parameters(), 
                    lr=HYPER_PARAMS['learning_rate'], 
                    correct_bias=False)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, 
                              sampler=train_sampler, 
                              batch_size=HYPER_PARAMS["training_batch_size"])

# optimizer = AdamW(bert_model.parameters(), 
#                   lr=HYPER_PARAMS['learning_rate'], 
#                   correct_bias=False)

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=HYPER_PARAMS['num_warmup_steps'], 
                                            num_training_steps=len(train_dataloader))


global_step = 0
tr_loss = 0.0
bert_model.zero_grad()
bert_model.train()

for epc in range(HYPER_PARAMS["num_training_epoch"]):
    print("Epoch #{}: \n".format(epc))
    epoch_iterator = tqdm.notebook.tqdm(train_dataloader, desc="Training Steps")
    avg_loss_over_epoch = []
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to('cuda') for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'labels': batch[3]}

        outputs = bert_model(**inputs)
        loss = outputs[0]  
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bert_model.parameters(), HYPER_PARAMS["max_grad_norm"])
        tr_loss += loss.item()

        optimizer.step()
        scheduler.step()
        bert_model.zero_grad()

Epoch #0: 




Epoch #1: 



KeyboardInterrupt: ignored

In [ ]:
import os
output_dir = '/content/drive/MyDrive/cis519_final/Bert-Base Models/elec_5cat_bert_base_adam/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

bert_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
config.save_pretrained(output_dir)

Load Model


In [ ]:
output_dir = '/content/drive/MyDrive/cis519_final/Bert-Base Models/elec_5cat_bert_base_adam/'
bert_model = BertForSequenceClassification.from_pretrained(output_dir, num_labels = 5)
tokenizer = BertTokenizer.from_pretrained(output_dir, num_labels = 5)

bert_model = bert_model.to("cuda")
  

In [ ]:
results = pd.DataFrame(columns=['Type', "Model", 'Optimizer', "Train Set", 'Test Set', 'LR', 'Dropout','F1', 'Accuracy', 'Recall', 'Precision' ])
optimizer_name = "ADAM"
test_set_name = "Elec"

In [ ]:

dev_sampler = SequentialSampler(dev_dataset)
dev_dataloader = DataLoader(dev_dataset, 
                            sampler=dev_sampler, 
                            batch_size=HYPER_PARAMS["eval_batch_size"])

predictions = None
out_label_ids = None

for batch in tqdm.notebook.tqdm(dev_dataloader, desc="Evaluating on Dev set..."):
    bert_model.eval()
    batch = tuple(t.to("cuda") for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'token_type_ids': batch[2],
              'labels': batch[3]}

    with torch.no_grad():
        outputs = bert_model(**inputs)
        logits = outputs[1] # This is 1x2 tensor, containing scores for both labels 
    if predictions is None:
        predictions = logits.detach().cpu().numpy()
        out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
        predictions = np.append(predictions, logits.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

# whichever label gets higher score, we will predict that label
predictions = np.argmax(predictions, axis=1)
# print(predictions)
# print(out_label_ids)


f1, acc, rec, prec = get_metrics(out_label_ids, predictions)
row = {'Type': type_name, 'Model':bert_model_type, 'Optimizer':optimizer_name, "Train Set": train_set_name, "Test Set": test_set_name, 'LR': HYPER_PARAMS['learning_rate'], 'Dropout':HYPER_PARAMS['dropout_prob'], 'F1': f1, 'Accuracy': acc, 'Recall': rec, 'Precision':prec}
print("The accuracy on dev set = {}".format(acc))
results = results.append(row, ignore_index = True)
results.to_csv(output_dir +"results.csv")

In [ ]:
results

Test on other dataset

In [ ]:
test_set_name = "All"

In [ ]:
dev_sampler = SequentialSampler(experiment_dataset)
dev_dataloader = DataLoader(experiment_dataset, 
                            sampler=dev_sampler, 
                            batch_size=HYPER_PARAMS["eval_batch_size"])

predictions = None
out_label_ids = None

for batch in tqdm.notebook.tqdm(dev_dataloader, desc="Evaluating on Experiment set..."):
    bert_model.eval()
    batch = tuple(t.to("cuda") for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'token_type_ids': batch[2],
              'labels': batch[3]}

    with torch.no_grad():
        outputs = bert_model(**inputs)
        logits = outputs[1] # This is 1x2 tensor, containing scores for both labels 
    if predictions is None:
        predictions = logits.detach().cpu().numpy()
        out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
        predictions = np.append(predictions, logits.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

# whichever label gets higher score, we will predict that label
predictions = np.argmax(predictions, axis=1)
# print(predictions)
# print(out_label_ids)


f1, acc, rec, prec = get_metrics(out_label_ids, predictions)
row = {'Type': type_name, 'Model':bert_model_type, 'Optimizer':optimizer_name, "Train Set": train_set_name, "Test Set": test_set_name, 'LR': HYPER_PARAMS['learning_rate'], 'Dropout':HYPER_PARAMS['dropout_prob'], 'F1': f1, 'Accuracy': acc, 'Recall': rec, 'Precision':prec, }
print("The accuracy on dev set = {}".format(acc))
results = results.append(row, ignore_index = True)
results.to_csv(output_dir +"results.csv")

In [ ]:
results

##MADGRAD

In [ ]:

bert_model_type = 'bert-base-uncased'  
bert_model = BertForSequenceClassification.from_pretrained(bert_model_type, num_labels = 5)
config = BertConfig.from_pretrained(bert_model_type)
tokenizer = BertTokenizer.from_pretrained(bert_model_type, num_labels = 5)

In [ ]:
HYPER_PARAMS = {
    "num_training_epoch": 3,
    "learning_rate": 5e-5,        
    "training_batch_size": 32,    
    "eval_batch_size": 8,
    "max_grad_norm": 1.0,
    "num_warmup_steps": 0.1,
    "dropout_prob": 0.1
}

In [ ]:
from madgrad import MADGRAD
torch.cuda.manual_seed(42)
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

bert_model.hidden_dropout_prob = HYPER_PARAMS['dropout_prob']
bert_model.to("cuda")
optimizer = MADGRAD(bert_model.parameters(), 
                    lr=HYPER_PARAMS['learning_rate'])
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, 
                              sampler=train_sampler, 
                              batch_size=HYPER_PARAMS["training_batch_size"])

# optimizer = AdamW(bert_model.parameters(), 
#                   lr=HYPER_PARAMS['learning_rate'], 
#                   correct_bias=False)

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=HYPER_PARAMS['num_warmup_steps'], 
                                            num_training_steps=len(train_dataloader))


global_step = 0
tr_loss = 0.0
bert_model.zero_grad()
bert_model.train()

for epc in range(HYPER_PARAMS["num_training_epoch"]):
    print("Epoch #{}: \n".format(epc))
    epoch_iterator = tqdm.notebook.tqdm(train_dataloader, desc="Training Steps")
    avg_loss_over_epoch = []
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to('cuda') for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'labels': batch[3]}

        outputs = bert_model(**inputs)
        loss = outputs[0]  
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bert_model.parameters(), HYPER_PARAMS["max_grad_norm"])
        tr_loss += loss.item()

        optimizer.step()
        scheduler.step()
        bert_model.zero_grad()

In [ ]:
# This is where we mounted your google drive. 
# You might need to re-mount it if your session was closed half way through
output_dir = '/content/drive/MyDrive/cis519_final/Bert-Base Models/elec_5cat_bert_base_madgrad/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

bert_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
config.save_pretrained(output_dir)


Load Model


In [ ]:
output_dir = '/content/drive/MyDrive/cis519_final/Bert-Base Models/elec_5cat_bert_base_madgrad/' 
bert_model = BertForSequenceClassification.from_pretrained(output_dir, num_labels = 5)
tokenizer = BertTokenizer.from_pretrained(output_dir, num_labels = 5)

bert_model = bert_model.to("cuda")

In [ ]:
results = pd.DataFrame(columns=['Type', "Model", 'Optimizer', "Train Set", 'Test Set', 'LR', 'Dropout','F1', 'Accuracy', 'Recall', 'Precision' ])
test_set_name = "Elec"
optimizer_name = "Madgrad"

In [ ]:
dev_sampler = SequentialSampler(dev_dataset)
dev_dataloader = DataLoader(dev_dataset, 
                            sampler=dev_sampler, 
                            batch_size=HYPER_PARAMS["eval_batch_size"])

predictions = None
out_label_ids = None

for batch in tqdm.notebook.tqdm(dev_dataloader, desc="Evaluating on Dev set..."):
    bert_model.eval()
    batch = tuple(t.to("cuda") for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'token_type_ids': batch[2],
              'labels': batch[3]}

    with torch.no_grad():
        outputs = bert_model(**inputs)
        logits = outputs[1] # This is 1x2 tensor, containing scores for both labels 
    if predictions is None:
        predictions = logits.detach().cpu().numpy()
        out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
        predictions = np.append(predictions, logits.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

# whichever label gets higher score, we will predict that label
predictions = np.argmax(predictions, axis=1)
# print(predictions)
# print(out_label_ids)


f1, acc, rec, prec = get_metrics(out_label_ids, predictions)
row = {'Type': type_name, 'Model':bert_model_type, 'Optimizer':optimizer_name, "Train Set": train_set_name, "Test Set": test_set_name, 'LR': HYPER_PARAMS['learning_rate'], 'Dropout':HYPER_PARAMS['dropout_prob'], 'F1': f1, 'Accuracy': acc, 'Recall': rec, 'Precision':prec, }
print("The accuracy on dev set = {}".format(acc))
results = results.append(row, ignore_index = True)
results.to_csv(output_dir+"results.csv")

In [ ]:
results

Test experiment

In [ ]:
test_set_name = "All"

In [ ]:
dev_sampler = SequentialSampler(experiment_dataset)
dev_dataloader = DataLoader(experiment_dataset, 
                            sampler=dev_sampler, 
                            batch_size=HYPER_PARAMS["eval_batch_size"])

predictions = None
out_label_ids = None

for batch in tqdm.notebook.tqdm(dev_dataloader, desc="Evaluating on Experiment set..."):
    bert_model.eval()
    batch = tuple(t.to("cuda") for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'token_type_ids': batch[2],
              'labels': batch[3]}

    with torch.no_grad():
        outputs = bert_model(**inputs)
        logits = outputs[1] # This is 1x2 tensor, containing scores for both labels 
    if predictions is None:
        predictions = logits.detach().cpu().numpy()
        out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
        predictions = np.append(predictions, logits.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

# whichever label gets higher score, we will predict that label
predictions = np.argmax(predictions, axis=1)
# print(predictions)
# print(out_label_ids)


f1, acc, rec, prec = get_metrics(out_label_ids, predictions)
row = {'Type': type_name, 'Model':bert_model_type, 'Optimizer':optimizer_name, "Train Set": train_set_name, "Test Set": test_set_name, 'LR': HYPER_PARAMS['learning_rate'], 'Dropout':HYPER_PARAMS['dropout_prob'], 'F1': f1, 'Accuracy': acc, 'Recall': rec, 'Precision':prec, }
print("The accuracy on dev set = {}".format(acc))
results = results.append(row, ignore_index = True)
results.to_csv(output_dir +"results.csv")

In [ ]:
results